In [ ]:
#load package
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import numpy as np
import seaborn as sns

from skimage import measure
from skimage.segmentation import clear_border
from skimage.morphology import skeletonize, binary_closing

In [ ]:
# A Framework to Access Handwritten Information within Large Digitized Paper Collections

In [ ]:
# load as a grayscale image, best used on scanned image
img_url = '/Users/livi/Git/Hand Writing Recognization/Example/images/SCN_0007-2.jpg'
img = cv2.imread(img_url,0)
plt.imshow(img)

In [ ]:
# binarized (background 0, line/text 1)
img = 1-img/255
plt.hist(img.reshape(-1,1)); plt.title('Intensity Distribution');plt.show()
#img = img[img>0.1]
#img = img[20:img.shape[0]-40,20:img.shape[1]-20]

#plt.figure(figsize=(16,8))
#plt.subplot(1,2,1)
#sns.heatmap(img); 

# morphological thinning process (ref 20) for character recognition. (likely to miss hlines)
#plt.subplot(1,2,2)
#img_thin = skeletonize(img)
#sns.heatmap(img_thin); 
#plt.show()

#Hough transformation
# rotate image by detecting long horizontal lines with Hough transform.
## histogram of angles of these lines, based on the angle to rotate
# after rotation, verticle and horizontal pixel histogram gves the location.
# we build one template, and fnd lines that match the template
#(dot product, or dynamic time warping). Horizontal/vertical can run separately.

In [ ]:
horizontal_sum = img.sum(axis=1)
vertical_sum = img.sum(axis=0)

# find loc read more than Q3 intensity.
h_diff = np.diff(horizontal_sum)
v_diff = np.diff(vertical_sum)
ax0_criteria, ax1_criteria = [np.mean(i)+1.5*np.std(i) for i in [h_diff, v_diff]]
#np.quantile(horizontal_sum,0.95), np.quantile(vertical_sum,0.95)

# image shape
print(img.shape)

plt.figure(figsize=(10,4))
# horizontal sum
plt.subplot(1,2,1)#; plt.plot(horizontal_sum); plt.hlines(ax0_criteria, 0, 300, 'r')
plt.plot(h_diff)
plt.axhline(np.mean(h_diff)+np.std(h_diff),c='r')
plt.axhline(np.mean(h_diff)-np.std(h_diff),c='r')
plt.axhline(ax0_criteria,c='g')
plt.title('Diff Horizontal Sum')
# vertical sumss
plt.subplot(1,2,2)#;plt.plot(vertical_sum); #plt.hlines(ax1_criteria, 0, 300, 'r')
plt.plot(v_diff)
plt.axhline(np.mean(v_diff)+np.std(v_diff),c='r')
plt.axhline(np.mean(v_diff)-np.std(v_diff),c='r')
plt.axhline(ax1_criteria,c='g')
plt.title('Vertical Sum')
plt.show()

In [ ]:
# draw cells
frame= np.zeros((img.shape[0]-1, img.shape[1]-1))
frame[h_diff>ax0_criteria,:]=1
frame[:,v_diff>ax1_criteria]=1
frame[h_diff<-1*ax0_criteria,:]=1
frame[:,v_diff<-1*ax1_criteria]=1

# merge lines that are closed to each other
frame_close = skeletonize(binary_closing(frame))
plt.figure(figsize=(16,8))
plt.subplot(1,2,1);sns.heatmap(1-frame)
plt.subplot(1,2,2);sns.heatmap(1-frame_close)
plt.show()

frame = frame_close

In [ ]:
frame = 255*(1-frame.astype(int))

## Filter img_0 based on Frame

In [ ]:
img_0 = 255-cv2.imread(img_url,0)
plt.figure(figsize=(16,8))
plt.subplot(1,2,1);sns.heatmap(img_0)
plt.subplot(1,2,2);sns.heatmap(clear_border(frame))
plt.show()

In [ ]:
frame = clear_border(frame)
frame_label = measure.label(frame)
smaller_regionNum=[]
area = {}
img_int = []

regions = measure.regionprops(frame_label)
for i, region in enumerate(regions):

    x1, y1, x2, y2= region.bbox
    img_int.append(np.sum(img_0[x1:x2, y1:y2]))
    
    area[i] = region.area
    if region.area<100:
        smaller_regionNum.append(region.label)
        #print(region.label,': ', region.area)

# reason for area < 100
print('# of Regions Smaller than 100: ', len(smaller_regionNum))
#smaller100= [i in smaller_regionNum for i in np.unique(frame_label)]

In [ ]:
sns.heatmap(frame_label)
plt.show()

In [ ]:
area_list = list(area.values())
CoefVar = np.std(area_list)/np.mean(area_list)
plt.hist(area.values())
plt.show()
CoefVar

In [ ]:
idx = [i for i,k in enumerate(area_list) if k>20000]
img_int = []
for i in idx:
    x1, y1, x2, y2= regions[i].bbox
    plt.subplot(121);plt.imshow(img_0[x1:x2, y1:y2]); plt.title(np.sum(img_0[x1:x2, y1:y2]))
    plt.subplot(122);plt.imshow(np.multiply(img_0[1:, 1:], frame_label==i+1))
    plt.show()



In [ ]:
_ = plt.hist(img_int, range=(0, 10000))
plt.show()